# Breve Exploração e Tratamento dos Dados

Importando bibliotecas

In [20]:
from pyspark.sql import SparkSession
import pandas as pd
from IPython.display import display, HTML
from pyspark.sql.functions import count, col, when, isnan
from pyspark.sql.types import DoubleType, IntegerType

Criando Sessão PySpark

In [21]:
!java -version
!pyspark --version

openjdk version "17.0.8.1" 2023-08-24
OpenJDK Runtime Environment (build 17.0.8.1+1-Ubuntu-0ubuntu122.04)
OpenJDK 64-Bit Server VM (build 17.0.8.1+1-Ubuntu-0ubuntu122.04, mixed mode, sharing)
Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.5.0
      /_/
                        
Using Scala version 2.12.18, OpenJDK 64-Bit Server VM, 17.0.8.1
Branch HEAD
Compiled by user ubuntu on 2023-09-09T01:53:20Z
Revision ce5ddad990373636e94071e7cef2f31021add07b
Url https://github.com/apache/spark
Type --help for more information.


In [22]:
spark = SparkSession.builder \
    .appName("Data-Process-Exploration") \
    .getOrCreate()


Lendo o DataFrame

In [23]:
path = "/home/jovyan/work/data/raw-data/creditcard_2023.csv"

df = spark.read.options(header = True).csv(path)

In [24]:
df.limit(10).toPandas().head()

,id,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-0.26064780489439815,-0.46964845005363426,2.4962660826315637,-0.08372391267814633,0.1296812361545678,0.7328982498449426,0.5190136179018007,-0.13000604758867731,0.7271592691096374,...,-0.11055167961012961,0.21760614382950005,-0.1347944948772723,0.1659591154312752,0.1262799761446219,-0.43482398075374323,-0.08123010860166756,-0.1510454864555771,17982.1,0
1,1,0.9850997342386376,-0.3560450929163436,0.5580563509382045,-0.4296539034065106,0.2771402629466986,0.4286045153379263,0.4064660422512956,-0.13311827417649086,0.3474518951767279,...,-0.19493596375310734,-0.6057609055521909,0.07946907582376135,-0.5773948740161742,0.1900897076741061,0.2965027040269081,-0.24805205857704948,-0.0645119229692018,6531.37,0
2,2,-0.26027161274297056,-0.9493846066454119,1.7285377761514877,-0.4579862888424837,0.07406165434922213,1.4194811432767418,0.7435110747693963,-0.09557601337146092,-0.2612966191422005,...,-0.0050202777531307315,0.7029063846645285,0.9450454906841043,-1.1546656286835624,-0.6055636614145458,-0.3128945478459945,-0.30025803544491,-0.24471822939465607,2513.54,0
3,3,-0.15215210191356718,-0.508958707673651,1.746840058804548,-1.090177941714601,0.2494857726542817,1.1433122633143087,0.5182685727677246,-0.0651299167095495,-0.2056976045189998,...,-0.1469271369017256,-0.038212457850069326,-0.21404818976618672,-1.8931311053959845,1.0039631060422864,-0.5159502587046078,-0.16531649030384224,0.048423625556267516,5384.44,0
4,4,-0.2068195207397724,-0.16528020377717972,1.5270526784614766,-0.44829266295851267,0.10612511416543663,0.5305488615008258,0.658849134344094,-0.21266001147475838,1.0499208392737625,...,-0.10698401767348882,0.729727387309421,-0.16166570362789645,0.31256100953490806,-0.4141161944011353,1.071125586437274,0.023711598569375818,0.419117267018846,14278.97,0


Esquema das colunas e tipos de dados **(tipo de dados estão incorretos)**

In [25]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- V1: string (nullable = true)
 |-- V2: string (nullable = true)
 |-- V3: string (nullable = true)
 |-- V4: string (nullable = true)
 |-- V5: string (nullable = true)
 |-- V6: string (nullable = true)
 |-- V7: string (nullable = true)
 |-- V8: string (nullable = true)
 |-- V9: string (nullable = true)
 |-- V10: string (nullable = true)
 |-- V11: string (nullable = true)
 |-- V12: string (nullable = true)
 |-- V13: string (nullable = true)
 |-- V14: string (nullable = true)
 |-- V15: string (nullable = true)
 |-- V16: string (nullable = true)
 |-- V17: string (nullable = true)
 |-- V18: string (nullable = true)
 |-- V19: string (nullable = true)
 |-- V20: string (nullable = true)
 |-- V21: string (nullable = true)
 |-- V22: string (nullable = true)
 |-- V23: string (nullable = true)
 |-- V24: string (nullable = true)
 |-- V25: string (nullable = true)
 |-- V26: string (nullable = true)
 |-- V27: string (nullable = true)
 |-- V28: string (nullabl

Número total de linhas

In [26]:
df.count()

568630

Número total de colunas

In [27]:
len(df.columns)

31

Estatísticas descritivas das colunas numéricas

In [28]:
display(HTML(df.describe().toPandas().to_html(notebook=True, escape=False)))

,summary,id,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,count,568630,568630,568630,568630,568630,568630,568630,568630,568630,568630,568630,568630,568630,568630,568630,568630,568630,568630,568630,568630,568630,568630,568630,568630,568630,568630,568630,568630,568630,568630,568630
1,mean,284314.5,-6.557741021875394E-16,9.276804372409095E-16,3.7427107295581524E-16,1.861758670600722E-15,9.564705197759722E-16,8.3811129157627E-16,2.0153057774543896E-15,-1.6074462748743345E-16,1.0268462770839033E-15,3.1349200982623837E-16,4.158567477286836E-16,-3.4036275352563026E-15,-3.1309214756880695E-16,-4.862325050366146E-16,3.7786983327269804E-17,1.3883217578019128E-15,8.253156993384644E-16,7.869289226250474E-16,5.71003303612077E-16,9.452743765678923E-16,-3.502793375099296E-16,-1.3615309865540072E-16,2.9839720960820204E-17,1.791382913292791E-16,1.0596349821932803E-16,1.6034476523000202E-16,1.2939542650480961E-15,-2.159256190129703E-17,12041.957634577875,0.5
2,stddev,164149.48612215227,1.0000008793075623,1.0000008793075632,1.0000008793075623,1.0000008793075634,1.000000879307563,1.000000879307562,1.0000008793075623,1.0000008793075625,1.0000008793075637,1.0000008793075634,1.0000008793075625,1.000000879307562,1.0000008793075628,1.000000879307563,1.0000008793075625,1.0000008793075628,1.0000008793075628,1.000000879307562,1.0000008793075637,1.000000879307563,1.0000008793075637,1.0000008793075632,1.0000008793075619,1.000000879307563,1.0000008793075634,1.000000879307563,1.0000008793075637,1.0000008793075637,6919.644449429182,0.5000004396537814
3,min,0,-0.00010567287917726666,-0.00010005440052480656,-0.00010319071647721545,-0.00010888279443271063,-0.00010918405857795954,-0.00011025638638988567,-0.00010325170959113737,-0.00010343807257444318,-0.00010031634603365231,-0.0001074946357181855,-0.0001005090662898387,-0.00010085380282659872,-0.00010829417452416742,-0.0001134719475575957,-0.00011376682306313075,-0.00010208091014732481,-0.0001245408526182491,-0.0001044321919020476,-0.00010376207825019841,-0.00010147243591341988,-0.0001003401113834677,-0.00010054568107380404,-0.00010451414670050604,-0.00010227400981829382,-0.00010610295907701043,-0.00010071245166617502,-0.00010372490785049114,-0.00010248147415389838,100.17,0
4,max,99999,9.66755846659791e-05,9.807491760704568e-05,9.973921146859824e-05,9.902109958138415e-05,9.998626740928714,9.937776357948239e-05,9.977545546051356e-05,9.993348351331015e-05,9.778179202592241,9.98326953234018,9.488928765055301e-05,9.329848308853592e-05,9.691548846570334e-05,9.972595565244005,9.501189790454801e-05,9.721005539279053e-05,9.366950604906313e-05,9.831594052051462e-05,9.918364557928006e-05,9.989279548011929,9.846622400245545e-05,9.858235304473647e-06,9.984671461141394e-05,9.97877756468287,9.99272107663856e-06,9.814711190836157e-05,9.960382327049274,9.994551647157735,9999.94,1


Verifiação de valores Nulos

In [29]:
df.select([count(when(col(c).isNull() | isnan(col(c)), c)).alias(c) for c in df.columns]).show()

+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+------+-----+
| id| V1| V2| V3| V4| V5| V6| V7| V8| V9|V10|V11|V12|V13|V14|V15|V16|V17|V18|V19|V20|V21|V22|V23|V24|V25|V26|V27|V28|Amount|Class|
+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+------+-----+
|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|     0|    0|
+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+------+-----+



Distribuição de Valores da coluna categórica

In [30]:
df.groupBy("Class").count().orderBy("count", ascending=False).show()

+-----+------+
|Class| count|
+-----+------+
|    0|284315|
|    1|284315|
+-----+------+



Converter os Tipos de Dados

In [31]:
numeric_cols = [f"v{i}" for i in range(1, 29)] + ["Amount"]
binary_cols = ["Class"]

In [32]:
df = df.select(
    [col(c).cast(DoubleType()).alias(c) for c in numeric_cols] +
    [col("Class").cast(IntegerType()).alias("Class")]
)

Exlcuindo coluna Id

In [33]:
df = df.drop("id")

Esquema das colunas e tipos de dados

In [34]:
df.printSchema()

root
 |-- v1: double (nullable = true)
 |-- v2: double (nullable = true)
 |-- v3: double (nullable = true)
 |-- v4: double (nullable = true)
 |-- v5: double (nullable = true)
 |-- v6: double (nullable = true)
 |-- v7: double (nullable = true)
 |-- v8: double (nullable = true)
 |-- v9: double (nullable = true)
 |-- v10: double (nullable = true)
 |-- v11: double (nullable = true)
 |-- v12: double (nullable = true)
 |-- v13: double (nullable = true)
 |-- v14: double (nullable = true)
 |-- v15: double (nullable = true)
 |-- v16: double (nullable = true)
 |-- v17: double (nullable = true)
 |-- v18: double (nullable = true)
 |-- v19: double (nullable = true)
 |-- v20: double (nullable = true)
 |-- v21: double (nullable = true)
 |-- v22: double (nullable = true)
 |-- v23: double (nullable = true)
 |-- v24: double (nullable = true)
 |-- v25: double (nullable = true)
 |-- v26: double (nullable = true)
 |-- v27: double (nullable = true)
 |-- v28: double (nullable = true)
 |-- Amount: double (nul

Lendo o DataFrame

In [35]:
df.limit(10).toPandas().head()

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,...,v21,v22,v23,v24,v25,v26,v27,v28,Amount,Class
0,-0.260648,-0.469648,2.496266,-0.083724,0.129681,0.732898,0.519014,-0.130006,0.727159,0.637735,...,-0.110552,0.217606,-0.134794,0.165959,0.126280,-0.434824,-0.081230,-0.151045,17982.10,0
1,0.985100,-0.356045,0.558056,-0.429654,0.277140,0.428605,0.406466,-0.133118,0.347452,0.529808,...,-0.194936,-0.605761,0.079469,-0.577395,0.190090,0.296503,-0.248052,-0.064512,6531.37,0
2,-0.260272,-0.949385,1.728538,-0.457986,0.074062,1.419481,0.743511,-0.095576,-0.261297,0.690708,...,-0.005020,0.702906,0.945045,-1.154666,-0.605564,-0.312895,-0.300258,-0.244718,2513.54,0
3,-0.152152,-0.508959,1.746840,-1.090178,0.249486,1.143312,0.518269,-0.065130,-0.205698,0.575231,...,-0.146927,-0.038212,-0.214048,-1.893131,1.003963,-0.515950,-0.165316,0.048424,5384.44,0
4,-0.206820,-0.165280,1.527053,-0.448293,0.106125,0.530549,0.658849,-0.212660,1.049921,0.968046,...,-0.106984,0.729727,-0.161666,0.312561,-0.414116,1.071126,0.023712,0.419117,14278.97,0


Salvando dataframe em arquivo .parquet

In [36]:
output_path = "/home/jovyan/work/data/clean-data/creditcard_2023.parquet"

df.write.parquet(output_path, mode="overwrite")

Finalizando a sessão Spark

In [37]:
spark.stop()